In [1]:
from datetime import datetime, timedelta
import os
import json
import requests
import hmac
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import time
from config import API_KEY
from client import FtxClient

ModuleNotFoundError: No module named 'config'

In [ ]:
url = 'https://ftx.com/api/markets'

#get BTCUSD spot price
base_currency = 'BTC'
quote_currency = 'USD'

#construct request url
request_url = f"{url}/{base_currency}/{quote_currency}"

btcusd_df = pd.DataFrame(requests.get(request_url).json())
btcusd_df['result']



ask                             24567.0
baseCurrency                        BTC
bid                             24566.0
change1h                       0.001917
change24h                      0.030884
changeBod                      0.007464
enabled                            True
futureType                         None
highLeverageFeeExempt              True
isEtfMarket                       False
largeOrderThreshold              3000.0
last                            24568.0
minProvideSize                   0.0001
name                            BTC/USD
postOnly                          False
price                           24567.0
priceHigh24h                    24618.0
priceIncrement                      1.0
priceLow24h                     23600.0
quoteCurrency                       USD
quoteVolume24h           486718102.1913
restricted                        False
sizeIncrement                    0.0001
type                               spot
underlying                         None


In [ ]:
def historical_bymonth(y, m, res="hourly"):
    """grabs historical BTCUSD 1H price from the FTX API by individual month

    Args:
        y (int): year
        m (int): month
        res (str): resolution options "hourly" or "daily", default is "hourly"
    """
    
    resolution = {
        
        "hourly":str(60*60),
        "daily":str(60*60*24)
    }
    
    
    start_date = datetime(year=y, month=m, day=1).timestamp()
    #end date wraps around across years
    end_date = (datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(seconds=15)).timestamp()
    
    historical = requests.get(
                f'{request_url}/candles?resolution={resolution[res]}&start_time={start_date}&end_time={end_date}'
                ).json()
    
    ls = historical['result']
    # df = pd.DataFrame(historical['result'])
    
    return ls

def get_multiple_months(y,m,n_months):
    """grabs multiple months of historical 1 hour data for BTCUSD FTX

    Args:
        y (int): starting year
        m (month): starting month
        n_months (int): number of months of data to be retrieved from the designated starting date
    """
    
    #maximum of 12 queries per second according to FTX docs
    MAX_PING_PER_SEC = 12
    
    ls = []
    
    
    for _ in range(n_months):
        ls.extend(historical_bymonth(y, m))
        
        if m % 12 == 0:
            y+=1
            
        m = (m % 12)+1

        #staggers every 1/12 seconds so that limit error cannot be triggered for any n_months requested
        time.sleep(1.0 / float(MAX_PING_PER_SEC))
    
    return ls

ls = get_multiple_months(2021,1,20)
df = pd.DataFrame(ls)
df

#convert date/time units
df['date'] = pd.to_datetime(
    df['time']/1000, unit='s', origin='unix'
)

#remove unnecessary columns
df.drop(['startTime','time'], axis=1, inplace=True)

df    
       
        

# df2 = historical_bymonth(2021,6,res="thirty")
# df2
# y = 2021
# m = 12
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1))
# print(datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1))
# print((datetime(y if m<12 else y+1, (m % 12)+1, 1) - timedelta(microseconds=1)).timestamp())
    

,open,high,low,close,volume,date
0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,2021-01-01 05:00:00
1,29240.0,29315.0,29127.5,29231.0,7.436047e+06,2021-01-01 06:00:00
2,29231.0,29241.5,28887.0,29168.0,8.430755e+06,2021-01-01 07:00:00
3,29168.0,29251.0,28928.5,29061.0,7.003067e+06,2021-01-01 08:00:00
4,29061.0,29379.5,29041.5,29275.0,5.764370e+06,2021-01-01 09:00:00
...,...,...,...,...,...,...
14129,24131.0,24259.0,24084.0,24251.0,1.905753e+07,2022-08-12 22:00:00
14130,24251.0,24465.0,24220.0,24416.0,2.741661e+07,2022-08-12 23:00:00
14131,24416.0,24505.0,24348.0,24496.0,1.508278e+07,2022-08-13 00:00:00
14132,24496.0,24618.0,24445.0,24569.0,3.618604e+07,2022-08-13 01:00:00


In [ ]:
#return SMA values
def sma(df, window=9):
    return df.rolling(window=window).mean()

#return EMA values
def ema(df, window=9):
    return df.ewm(span=window, adjust=False).mean()


def liq(direction, lev=25, maint = 0.005):
    """Calculates liquidation price given the entry price (EMA9 is used here to track price over average of ~1 session where one session = 8hrs
    #Liquidation calculation provided by exchange
    Liquidation Price = ((Price*Leverage)/(Leverage+1 - (Maintenance Margin * Leverage))) 
    

    Args:
        lev (int): leverage used (options: 10,25,50,100)
        direction (string): 'long' or 'short' position
        maint (float): maintanance margin (0.005 for BTCUSD and 0.01 for ETHUSD)
        

    Returns:
        ls: list of liquidation prices 
    """
    ls = []
    ema9 = df['EMA9']
    
    if maint == 0.005:
        if direction == 'long':
            
            if lev == 10:
                ls = ((ema9*10)/(10+1-(0.005*10)))
            elif lev == 25:
                ls = ((ema9*25)/(25+1-(0.005*25)))
            elif lev == 50:
                ls = ((ema9*50)/(50+1-(0.005*50)))
            else:
                ls = ((ema9*99)/(99+1-(0.005*99)))
            
        else:
            
            if lev == 10:
                ls = ((ema9*10)/(10-1+(0.005*10)))
            elif lev == 25:
                ls = ((ema9*25)/(25-1+(0.005*25)))
            elif lev == 50:
                ls = ((ema9*50)/(50-1+(0.005*50)))
            else:
                ls = ((ema9*99)/(99-1+(0.005*99)))
    else:
        pass
                
    
    
    return ls

#add ohlc4 column
df['ohlc4'] = (df['open']+df['high']+df['low']+df['close'])/4

#add SMA column 
df['SMA48'] = sma(df['ohlc4'], window=48)
df['SMA9'] = sma(df['ohlc4'], window=9)


#add EMA columns
df['EMA9'] = ema(df['ohlc4'], window=9)
df['EMA48'] = ema(df['ohlc4'], window=48)
ema9 = df['EMA9']


# add liquidation value columns 10,25,50,100x 
df['10x short'] = liq('short',10)
df['25x short'] = liq('short',25)
df['50x short'] = liq('short',50)
df['100x short'] = liq('short',100)

df['10x long'] = liq('long',10)
df['25x long'] = liq('long',25)
df['50x long'] = liq('long',50)
df['100x long'] = liq('long',100)


df 
data = df


In [ ]:
#create hour column
df['hour']=df.date.dt.hour
df['hour']=df['hour'].astype(float)

In [ ]:

#date for volatility
data = df.set_index('date')

high_low = data['high']-data['low']
high_cp = np.abs(data['high']-data['close'].shift())
low_cp = np.abs(data['low']-data['close'].shift())

df_volatility = pd.concat([high_low, high_cp, low_cp], axis =1)

#calc true range
true_range = np.max(df_volatility, axis =1)

#calc avg true range
average_true_range = true_range.rolling(14).mean()

data['average_true_range']=average_true_range


data = data.reset_index(drop=False)

data = data.rename(columns = {"volume":"volume(BTC)"})

data = data[['date','hour','open','high','low','close','volume(BTC)','ohlc4', 'SMA48', 'average_true_range',
             'EMA9','EMA48','10x short','25x short', '50x short', '100x short', '10x long', '25x long', '50x long','100x long']]
data

,date,hour,open,high,low,close,volume(BTC),ohlc4,SMA48,average_true_range,EMA9,EMA48,10x short,25x short,50x short,100x short,10x long,25x long,50x long,100x long
0,2021-01-01 05:00:00,5.0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,29240.750,NaN,NaN,29240.750000,29240.750000,32310.220994,30301.295337,29686.040609,29390.672115,26703.881279,28251.932367,28808.620690,29092.349631
1,2021-01-01 06:00:00,6.0,29240.0,29315.0,29127.5,29231.0,7.436047e+06,29228.375,NaN,NaN,29238.275000,29240.244898,32307.486188,30298.730570,29683.527919,29388.184426,26701.621005,28249.541063,28806.182266,29089.887192
2,2021-01-01 07:00:00,7.0,29231.0,29241.5,28887.0,29168.0,8.430755e+06,29131.875,NaN,NaN,29216.995000,29235.821637,32283.972376,30276.678756,29661.923858,29366.795320,26682.187215,28228.980676,28785.216749,29068.715190
3,2021-01-01 08:00:00,8.0,29168.0,29251.0,28928.5,29061.0,7.003067e+06,29102.125,NaN,NaN,29194.021000,29230.364631,32258.586740,30252.871503,29638.600000,29343.703528,26661.206393,28206.783575,28762.582266,29045.857786
4,2021-01-01 09:00:00,9.0,29061.0,29379.5,29041.5,29275.0,5.764370e+06,29189.250,NaN,NaN,29193.066800,29228.686483,32257.532376,30251.882694,29637.631269,29342.744436,26660.334977,28205.861643,28761.642167,29044.908429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,2022-08-12 22:00:00,22.0,24131.0,24259.0,24084.0,24251.0,1.905753e+07,24181.250,24175.859375,185.142857,24089.961213,24010.637058,26618.741672,24963.690376,24456.813414,24213.474390,21999.964578,23275.324843,23733.951934,23967.701724
14130,2022-08-12 23:00:00,23.0,24251.0,24465.0,24220.0,24416.0,2.741661e+07,24338.000,24184.932292,185.857143,24139.568970,24023.998811,26673.556873,25015.097379,24507.176620,24263.336495,22045.268466,23323.255044,23782.826572,24017.057716
14131,2022-08-13 00:00:00,0.0,24416.0,24505.0,24348.0,24496.0,1.508278e+07,24441.250,24193.890625,176.714286,24199.905176,24041.029472,26740.226714,25077.621944,24568.431651,24323.982054,22100.370024,23381.550895,23842.271110,24077.087709
14132,2022-08-13 01:00:00,1.0,24496.0,24618.0,24445.0,24569.0,3.618604e+07,24532.000,24199.479167,173.357143,24266.324141,24061.069085,26813.617835,25146.449887,24635.862072,24390.741560,22161.026613,23445.723808,23907.708513,24143.169589


In [ ]:
cols = ['ohlc4', 'SMA48', 'average_true_range','EMA9','EMA48','10x short','25x short', '50x short', '100x short', '10x long', '25x long', '50x long','100x long']
data[cols] = data[cols].round(2)
data

,date,hour,open,high,low,close,volume,ohlc4,SMA48,average_true_range,EMA9,EMA48,10x short,25x short,50x short,100x short,10x long,25x long,50x long,100x long
0,2021-01-01 05:00:00,5.0,29280.5,29296.0,29146.5,29240.0,5.523795e+06,29240.75,NaN,NaN,29240.75,29240.75,32310.22,30301.30,29686.04,29390.67,26703.88,28251.93,28808.62,29092.35
1,2021-01-01 06:00:00,6.0,29240.0,29315.0,29127.5,29231.0,7.436047e+06,29228.38,NaN,NaN,29238.28,29240.24,32307.49,30298.73,29683.53,29388.18,26701.62,28249.54,28806.18,29089.89
2,2021-01-01 07:00:00,7.0,29231.0,29241.5,28887.0,29168.0,8.430755e+06,29131.88,NaN,NaN,29217.00,29235.82,32283.97,30276.68,29661.92,29366.80,26682.19,28228.98,28785.22,29068.72
3,2021-01-01 08:00:00,8.0,29168.0,29251.0,28928.5,29061.0,7.003067e+06,29102.12,NaN,NaN,29194.02,29230.36,32258.59,30252.87,29638.60,29343.70,26661.21,28206.78,28762.58,29045.86
4,2021-01-01 09:00:00,9.0,29061.0,29379.5,29041.5,29275.0,5.764370e+06,29189.25,NaN,NaN,29193.07,29228.69,32257.53,30251.88,29637.63,29342.74,26660.33,28205.86,28761.64,29044.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,2022-08-12 22:00:00,22.0,24131.0,24259.0,24084.0,24251.0,1.905753e+07,24181.25,24175.86,185.14,24089.96,24010.64,26618.74,24963.69,24456.81,24213.47,21999.96,23275.32,23733.95,23967.70
14130,2022-08-12 23:00:00,23.0,24251.0,24465.0,24220.0,24416.0,2.741661e+07,24338.00,24184.93,185.86,24139.57,24024.00,26673.56,25015.10,24507.18,24263.34,22045.27,23323.26,23782.83,24017.06
14131,2022-08-13 00:00:00,0.0,24416.0,24505.0,24348.0,24496.0,1.508278e+07,24441.25,24193.89,176.71,24199.91,24041.03,26740.23,25077.62,24568.43,24323.98,22100.37,23381.55,23842.27,24077.09
14132,2022-08-13 01:00:00,1.0,24496.0,24618.0,24445.0,24569.0,3.618604e+07,24532.00,24199.48,173.36,24266.32,24061.07,26813.62,25146.45,24635.86,24390.74,22161.03,23445.72,23907.71,24143.17


In [ ]:
#save as csv
# data.to_csv("resources/2021_2022_FTX_USDBTC_1H.csv", index=False, header=True)